In [ ]:
import numpy as np
import time
import ctypes
from utility.path_tracking import search, \
smooth, transform_points_from_image2real, \
transform2robot_frame, is_near, \
get_distance, \
transform_pos_angle, pid, \
send_path_4_drawing, \
pioneer_robot_model
from vision.tracker import Tracker, get_marker_object, get_markers, CAM_MAT, CAM_DIST
from robot.evolved_robot import EvolvedRobot
import vrep.vrep as vrep

In [ ]:
vision_thread = Tracker(mid=5,
                        transform=None,
                        mid_aux=0,
                        video_source=-1,
                        capture=False,
                        show=True,
                        debug=False,
                       )

In [ ]:
vision_thread.start()

In [ ]:
vrep.simxFinish(-1)
clientID = vrep.simxStart(
    '127.0.0.1',
    19997,
    True,
    True,
    5000,
    5)

if clientID == -1:
    print('Failed connecting to remote API server')
    print('Program ended')

if (vrep.simxStartSimulation(clientID, vrep.simx_opmode_blocking) == -1):
    print('Failed to start the simulation\n')
    print('Program ended\n')


OP_MODE = vrep.simx_opmode_oneshot_wait
robot = EvolvedRobot('thymio-II', clientID, None, OP_MODE, None)

robot_m = get_marker_object(7)
while robot_m.realxy() is None:
    robot_m = get_marker_object(7)

In [ ]:
if robot_m.orientation() > np.pi:
    angle = (2*np.pi - robot_m.orientation())
else:
    angle = -(robot_m.orientation())

In [ ]:
# set position of the robot in simulator
position, orientation = transform_pos_angle(robot_m.realxy()[:2], angle)
robot.v_set_pos_angle(position, orientation)